In [1]:
from util_for_test import make_unique_eqs, finder, check
import numpy as np
max_n_op = 4
upper = 1.5
lower = 0.5
len_x = 40
how_many_to_save = 500000
x = np.random.uniform(lower, upper, (max_n_op + 1, len_x))
calced_x, calced_eq, arr_columns = make_unique_eqs(x, max_n_op, how_many_to_save=how_many_to_save, is_print=True, log_interval=120)

2025-04-06 10:56:47,047 : numba=0.61.0, numpy=2.1.3
2025-04-06 10:56:47,047 : OPT=_OptLevel(3), THREADING_LAYER=default
2025-04-06 10:56:47,048 : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2025-04-06 10:56:47,048 : SIS
2025-04-06 10:56:47,048 : num_threads=8, how_many_to_save=500000, 
2025-04-06 10:56:47,048 : how_many_to_save_per_1_core=63814, 
2025-04-06 10:56:47,048 : max_n_op=4, model_score=debug_1d, 
2025-04-06 10:56:47,049 : x.shape=(5, 40), is_use_1=True
2025-04-06 10:56:47,049 : use_binary_op=[-1, -2, -3, -4], 
2025-04-06 10:56:47,049 : use_unary_op=[]
2025-04-06 10:56:47,049 : units=[0] , [0] , [0] , [0] , [0]
2025-04-06 10:56:47,050 : compiling
2025-04-06 10:57:04,906 : END, compile
2025-04-06 10:57:04,908 :   n_op=1
2025-04-06 10:57:05,146 :     binary_op n_op1:n_op2 = 0:0,  loop:144
2025-04-06 10:57:05,163 :       time : 0:00:00.015930
2025-04-06 10:57:05,163 :     END, time=0:00:00.255188
2025-04-06 10:57:05,163 :   n_op=2
2025-04-06 10:57:05,164 :     binary_op n_op

In [2]:
calced_eq.shape

(418697, 9)

In [2]:
target_eq = np.array([1,2,3,-4,4,-3,1,-2,-4])
finder(target_eq, x=x, calced_x=calced_x, calced_eq=calced_eq)

finding : 
same :  [ 1  2  3 -4  4 -3  1 -2 -4]
done : make random_x


In [3]:
n_random = 1000000
find = ["not_calced", "many_calc"]# ["not_calced"]
check(n_random, x, calced_x, calced_eq, find, log_interval=5)

many calc :  2 [   1    1    5   -1    4   -3   -3 -100 -100]
[ 4  4 -1  1  5 -1 -3  1 -3]
[   1    5   -1    4   -3    1   -3 -100 -100]


False

In [ ]:
from util_for_test import finder
import numpy as np
target_eq = np.array([1, 1, -3, 2, -4, 2, -2])
finder(target_eq)

In [ ]:
[ 1  2  2 -3  1 -4  1 -2 -4]

In [ ]:
not calced :  [ 2  2  4  1  5 -4 -3 -2 -4] 0.09279815643807351
not calced :  [ 5  5  3  4  2 -4 -3 -1 -4] 1.622520828752577
not calced :  [ 4  5  1 -3  2 -4  4 -2 -4] 0.05453123325323784
not calced :  [ 5  5  1  4  2 -4 -3 -2 -4] 0.157141080172706
not calced :  [ 5  5  4  2  1 -4 -3 -2 -4] 0.05453123325323879
not calced :  [ 5  5  1  5 -4  1 -3 -1 -4] 1.5912550351971864
not calced :  [ 3  2  1 -4  5 -3  3 -2 -4] 0.1057777231470847
not calced :  [ 3  4  4 -3  3 -4  3 -2 -4] 0.0704404922652119
not calced :  [ 4  4  3  1 -4  2 -3 -1 -4] 0.8468567934189181

In [ ]:
from numba import njit, prange
import numpy as np

@njit(parallel=True, error_model="numpy")
def check_multi_or(arr_len, num_threads=8):
    save = np.zeros(arr_len, dtype="bool")
    a = np.zeros((num_threads, arr_len), dtype="bool")
    for thread_id in prange(num_threads):
        indexes = np.random.permutation(np.arange(arr_len))[:arr_len//num_threads]
        for i in indexes:
            a[thread_id, i] = True
            save[i] = True
    b = np.zeros(arr_len, dtype="bool")
    for thread_id in range(num_threads):
        b|=a[thread_id]
    return np.all(save == b)
check_multi_or(1000000, num_threads=18)

In [ ]:
from numba import njit, prange
import numpy as np

@njit(parallel=True, error_model="numpy")
def check_multi_plus(arr_len, num_threads=8):
    save = np.zeros(arr_len, dtype="int64")
    a = np.zeros((num_threads, arr_len), dtype="int64")
    for thread_id in prange(num_threads):
        indexes = np.random.permutation(np.arange(arr_len))[:arr_len//num_threads + 1]
        for i in indexes:
            a[thread_id, i] += 1
            save[i] += 1
    b = np.zeros(arr_len, dtype="int64")
    for thread_id in range(num_threads):
        b+=a[thread_id]
    #return np.all(save == b)
    return save, b, np.all(save == b)
check_multi_plus(100, num_threads=18)